In [1]:
import os 


In [2]:
%pwd

'/Users/ravina/Desktop/project_mlflow/research'

from above we cn see that we are in the reasearch folder, but i want to go back to the folder /Users/ravina/Desktop/project_mlflow
so that i can do everything in the project folder.
so to back to one folder back following is the required command


In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/ravina/Desktop/project_mlflow'

In [5]:
#entity: return type of a function
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)  #this is not python class but dataclass, here you can define the veriables without using self keyword.
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path



In [6]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories




In [47]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.get('data_ingestion')
        if config is None:
            raise ValueError("Data Ingestion configuration is missing or invalid.")

        create_directories([config.get('root_dir')])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [48]:
import os 
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [49]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [50]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-09 14:10:20,312:INFO:yaml file:config/config.yaml loaded successfully]
[2024-01-09 14:10:20,314:INFO:yaml file:params.yaml loaded successfully]
[2024-01-09 14:10:20,315:INFO:yaml file:Schema.yaml loaded successfully]
[2024-01-09 14:10:20,316:INFO:created directory at:artifacts]
[2024-01-09 14:10:20,316:INFO:created directory at:artifacts/data_ingestion]
[2024-01-09 14:10:21,263:INFO:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 22286
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "b8569752c038a929d474d6f28ab0b8c15a4b97751d5b19f4b5f469dc60eb50a2"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3C9A:3F2FBB:86424E:8C99FD:659D0674
Accept-Ranges: bytes
Date: Tue, 09 Jan 2024 08:40:21 GMT
Via: 1.1 varnish
X-Served-By: cache-lhr7359-LH